In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Telomere_Length"
cohort = "GSE52237"

# Input paths
in_trait_dir = "../../input/GEO/Telomere_Length"
in_cohort_dir = "../../input/GEO/Telomere_Length/GSE52237"

# Output paths
out_data_file = "../../output/preprocess/Telomere_Length/GSE52237.csv"
out_gene_data_file = "../../output/preprocess/Telomere_Length/gene_data/GSE52237.csv"
out_clinical_data_file = "../../output/preprocess/Telomere_Length/clinical_data/GSE52237.csv"
json_path = "../../output/preprocess/Telomere_Length/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
# 1. Let's first list the directory contents to understand what files are available
import os

print("Files in the cohort directory:")
files = os.listdir(in_cohort_dir)
print(files)

# Adapt file identification to handle different naming patterns
soft_files = [f for f in files if 'soft' in f.lower() or '.soft' in f.lower() or '_soft' in f.lower()]
matrix_files = [f for f in files if 'matrix' in f.lower() or '.matrix' in f.lower() or '_matrix' in f.lower()]

# If no files with these patterns are found, look for alternative file types
if not soft_files:
    soft_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]
if not matrix_files:
    matrix_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]

print("Identified SOFT files:", soft_files)
print("Identified matrix files:", matrix_files)

# Use the first files found, if any
if len(soft_files) > 0 and len(matrix_files) > 0:
    soft_file = os.path.join(in_cohort_dir, soft_files[0])
    matrix_file = os.path.join(in_cohort_dir, matrix_files[0])
    
    # 2. Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # 3. Obtain the sample characteristics dictionary from the clinical dataframe
    sample_characteristics_dict = get_unique_values_by_row(clinical_data)
    
    # 4. Explicitly print out all the background information and the sample characteristics dictionary
    print("\nBackground Information:")
    print(background_info)
    print("\nSample Characteristics Dictionary:")
    print(sample_characteristics_dict)
else:
    print("No appropriate files found in the directory.")


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
import pandas as pd
import numpy as np
import os

# 1. Gene Expression Data Availability
# From the background information, this study involves gene expression related to aging
# and telomere length, which is relevant to our Telomere_Length trait
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# After careful examination of the sample characteristics and background information:
# The "cilia length" in row 1 appears to be related to telomere measurements
# as the study specifically mentions telomere length measurements
trait_row = 1  # Row 1 contains "cilia length" which likely represents telomere length

# For age and gender, there's no information in the sample characteristics
age_row = None
gender_row = None

# 2.2 Data Type Conversion functions
def convert_trait(x):
    """Convert telomere/cilia length data to continuous values."""
    if pd.isna(x):
        return None
    try:
        # Extract value after colon if present
        if ":" in str(x):
            value = str(x).split(":")[1].strip()
            return float(value)
        return float(x)
    except:
        return None

def convert_age(x):
    """Convert age data to continuous values."""
    if pd.isna(x):
        return None
    try:
        # Extract value after colon if present
        if ":" in str(x):
            value = str(x).split(":")[1].strip()
            return float(value)
        return float(x)
    except:
        return None

def convert_gender(x):
    """Convert gender data to binary values: 0 for female, 1 for male."""
    if pd.isna(x):
        return None
    
    x_lower = str(x).lower()
    
    # Extract value after colon if present
    if ":" in x_lower:
        value = x_lower.split(":")[1].strip()
    else:
        value = x_lower.strip()
    
    if "female" in value or "f" == value:
        return 0
    elif "male" in value or "m" == value:
        return 1
    else:
        return None

# 3. Save Metadata
# Based on our identification of telomere length data in row 1
is_trait_available = trait_row is not None

# Save initial filtering results
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we proceed with clinical feature extraction
if trait_row is not None:
    # Assuming clinical_data was already loaded in a previous step
    matrix_file = os.path.join(in_cohort_dir, "GSE52237_series_matrix.txt.gz")
    clinical_data = parse_geo_matrix(matrix_file)
    
    # Select and process clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the selected clinical features
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Save the clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Initial Data Loading

In [ ]:
# 1. Let's first list the directory contents to understand what files are available
import os

print("Files in the cohort directory:")
files = os.listdir(in_cohort_dir)
print(files)

# Adapt file identification to handle different naming patterns
soft_files = [f for f in files if 'soft' in f.lower() or '.soft' in f.lower() or '_soft' in f.lower()]
matrix_files = [f for f in files if 'matrix' in f.lower() or '.matrix' in f.lower() or '_matrix' in f.lower()]

# If no files with these patterns are found, look for alternative file types
if not soft_files:
    soft_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]
if not matrix_files:
    matrix_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]

print("Identified SOFT files:", soft_files)
print("Identified matrix files:", matrix_files)

# Use the first files found, if any
if len(soft_files) > 0 and len(matrix_files) > 0:
    soft_file = os.path.join(in_cohort_dir, soft_files[0])
    matrix_file = os.path.join(in_cohort_dir, matrix_files[0])
    
    # 2. Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # 3. Obtain the sample characteristics dictionary from the clinical dataframe
    sample_characteristics_dict = get_unique_values_by_row(clinical_data)
    
    # 4. Explicitly print out all the background information and the sample characteristics dictionary
    print("\nBackground Information:")
    print(background_info)
    print("\nSample Characteristics Dictionary:")
    print(sample_characteristics_dict)
else:
    print("No appropriate files found in the directory.")


### Step 4: Gene Data Extraction

In [ ]:
# Use the helper function to get the proper file paths
soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)

# Extract gene expression data
try:
    gene_data = get_genetic_data(matrix_file_path)
    
    # Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
    
    # Print shape to understand the dataset dimensions
    print(f"\nGene expression data shape: {gene_data.shape}")
    
except Exception as e:
    print(f"Error extracting gene data: {e}")


### Step 5: Gene Identifier Review

In [ ]:
# Examining the gene identifiers shown in the output
# These identifiers (like '1007_s_at', '1053_at') appear to be Affymetrix probe IDs
# They are not standard human gene symbols (which would look like BRCA1, TP53, etc.)
# These probe IDs need to be mapped to human gene symbols for biological interpretation

requires_gene_mapping = True


### Step 6: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
try:
    # Use the correct variable name from previous steps
    gene_annotation = get_gene_annotation(soft_file_path)
    
    # 2. Preview the gene annotation dataframe
    print("Gene annotation preview:")
    print(preview_df(gene_annotation))
    
except UnicodeDecodeError as e:
    print(f"Unicode decoding error: {e}")
    print("Trying alternative approach...")
    
    # Read the file with Latin-1 encoding which is more permissive
    import gzip
    import pandas as pd
    
    # Manually read the file line by line with error handling
    data_lines = []
    with gzip.open(soft_file_path, 'rb') as f:
        for line in f:
            # Skip lines starting with prefixes we want to filter out
            line_str = line.decode('latin-1')
            if not line_str.startswith('^') and not line_str.startswith('!') and not line_str.startswith('#'):
                data_lines.append(line_str)
    
    # Create dataframe from collected lines
    if data_lines:
        gene_data_str = '\n'.join(data_lines)
        gene_annotation = pd.read_csv(pd.io.common.StringIO(gene_data_str), sep='\t', low_memory=False)
        print("Gene annotation preview (alternative method):")
        print(preview_df(gene_annotation))
    else:
        print("No valid gene annotation data found after filtering.")
        gene_annotation = pd.DataFrame()
    
except Exception as e:
    print(f"Error extracting gene annotation data: {e}")
    gene_annotation = pd.DataFrame()


### Step 7: Gene Identifier Mapping

In [ ]:
# 1. Determine which columns in the gene annotation dataframe to use for mapping
# From the preview, we can see:
# - 'ID' column contains probe IDs like '1007_s_at' that match gene expression data indices
# - 'Gene Symbol' column contains the actual gene symbols we want to map to

# 2. Get gene mapping dataframe using the identified columns
gene_mapping = get_gene_mapping(gene_annotation, 'ID', 'Gene Symbol')
print(f"Gene mapping dataframe shape: {gene_mapping.shape}")
print("Gene mapping preview:")
print(gene_mapping.head())

# 3. Convert probe-level measurements to gene expression data
# Apply the gene mapping to convert from probe IDs to gene symbols
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Gene expression data after mapping - shape: {gene_data.shape}")
print("Gene expression data preview (first 5 genes):")
print(gene_data.head())

# Normalize gene symbols to ensure consistency
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene expression data after symbol normalization - shape: {gene_data.shape}")
print("Gene expression data preview after normalization (first 5 genes):")
print(gene_data.head())


### Step 8: Data Normalization and Linking

In [ ]:
# 1. Save the normalized gene expression data
print(f"Normalized gene data shape: {gene_data.shape}")
print(f"First few normalized gene symbols: {list(gene_data.index[:10])}")

# Make sure directory exists
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
# Save gene expression data
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")

# 2. Create clinical dataframe using geo_select_clinical_features
# Re-extract background information and clinical data from the matrix file
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file_path, background_prefixes, clinical_prefixes)

# Define conversion functions for telomere length and smoking status
def convert_trait(x):
    """Convert telomere/cilia length data to continuous values."""
    if pd.isna(x):
        return None
    try:
        # Extract value after colon if present
        if ":" in str(x):
            value = str(x).split(":")[1].strip()
            return float(value)
        return float(x)
    except:
        return None

def convert_smoking(x):
    """Convert smoking status to binary (0: nonsmoker, 1: smoker)."""
    if pd.isna(x):
        return None
    x_lower = str(x).lower()
    if ":" in x_lower:
        value = x_lower.split(":")[1].strip()
    else:
        value = x_lower.strip()
    
    if "nonsmoker" in value:
        return 0
    elif "smoker" in value:
        return 1
    else:
        return None

# Extract clinical features - trait_row=1 for telomere length, row 0 for smoking status
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=1,  # Corresponds to "cilia length" in the sample characteristics
    convert_trait=convert_trait,
    gender_row=None,  # No gender data available
    convert_gender=None,
    age_row=0,  # Using smoking status as an additional feature
    convert_age=convert_smoking  # Reusing age slot for smoking status
)

# Save clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")
print("Clinical data preview:")
print(selected_clinical_df.head())

# 3. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, gene_data)
print(f"Linked data shape: {linked_data.shape}")
print(f"First few columns of linked data: {linked_data.columns[:10].tolist()}")

# 4. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 5. Determine whether the trait and demographic features are biased
is_trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
print(f"Is trait biased: {is_trait_biased}")
print(f"Linked data shape after removing biased features: {linked_data.shape}")

# 6. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True,  
    is_trait_available=True,
    is_biased=is_trait_biased, 
    df=linked_data,
    note="Dataset contains telomere length measurements and gene expression data from smokers and nonsmokers, studying how smoking affects aging of the small airway epithelium."
)

# 7. Save the linked data if it's usable
print(f"Data quality check result: {'Usable' if is_usable else 'Not usable'}")
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data not saved due to quality issues.")


### Step 9: Gene Data Extraction

In [ ]:
# Use the helper function to get the proper file paths
soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)

# Extract gene expression data
try:
    gene_data = get_genetic_data(matrix_file_path)
    
    # Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
    
    # Print shape to understand the dataset dimensions
    print(f"\nGene expression data shape: {gene_data.shape}")
    
except Exception as e:
    print(f"Error extracting gene data: {e}")


### Step 10: Gene Identifier Mapping

In [ ]:
# Get the proper file paths for SOFT and matrix files
soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Using SOFT file: {soft_file_path}")

# Extract gene annotation data from the SOFT file
gene_annotation = get_gene_annotation(soft_file_path)
print(f"Gene annotation data shape: {gene_annotation.shape}")

# Get gene mapping dataframe using the identified columns
# 'ID' column contains probe IDs like '1007_s_at' that match gene expression data indices
# 'Gene Symbol' column contains the actual gene symbols we want to map to
gene_mapping = get_gene_mapping(gene_annotation, 'ID', 'Gene Symbol')
print(f"Gene mapping dataframe shape: {gene_mapping.shape}")
print("Gene mapping preview:")
print(gene_mapping.head())

# Convert probe-level measurements to gene expression data
# Apply the gene mapping to convert from probe IDs to gene symbols
mapped_gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Gene expression data after mapping - shape: {mapped_gene_data.shape}")
print("Gene expression data preview (first 5 genes):")
print(mapped_gene_data.head())

# Normalize gene symbols to ensure consistency
gene_data = normalize_gene_symbols_in_index(mapped_gene_data)
print(f"Gene expression data after symbol normalization - shape: {gene_data.shape}")
print("Gene expression data preview after normalization (first 5 genes):")
print(gene_data.head())


### Step 11: Gene Data Extraction

In [ ]:
# Use the helper function to get the proper file paths
soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)

# Extract gene expression data
try:
    gene_data = get_genetic_data(matrix_file_path)
    
    # Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
    
    # Print shape to understand the dataset dimensions
    print(f"\nGene expression data shape: {gene_data.shape}")
    
except Exception as e:
    print(f"Error extracting gene data: {e}")


### Step 12: Gene Identifier Mapping

In [ ]:
# Get the proper file paths for SOFT and matrix files
soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Using SOFT file: {soft_file_path}")

# 1. Extract gene annotation data from the SOFT file
gene_annotation = get_gene_annotation(soft_file_path)
print(f"Gene annotation data shape: {gene_annotation.shape}")

# Examine columns to see which ones to use for mapping
print("Columns in gene annotation data:")
print(gene_annotation.columns.tolist())

# 2. Get gene mapping dataframe using the identified columns
# From the previous output, 'ID' column contains probe IDs that match gene expression data indices
# 'Gene Symbol' column contains the actual gene symbols we want to map to
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Gene Symbol')
print(f"Gene mapping dataframe shape: {gene_mapping.shape}")
print("Gene mapping preview:")
print(gene_mapping.head())

# 3. Convert probe-level measurements to gene expression data
# Apply the gene mapping to convert from probe IDs to gene symbols
gene_data_mapped = apply_gene_mapping(gene_data, gene_mapping)
print(f"Gene expression data after mapping - shape: {gene_data_mapped.shape}")
print("Gene expression data preview (first 5 genes):")
print(gene_data_mapped.head())

# Normalize gene symbols to ensure consistency
gene_data = normalize_gene_symbols_in_index(gene_data_mapped)
print(f"Gene expression data after symbol normalization - shape: {gene_data.shape}")
print("Gene expression data preview after normalization (first 5 genes):")
print(gene_data.head())


### Step 13: Data Normalization and Linking

In [ ]:
# 1. Normalize gene symbols in the obtained gene expression data
# Note: We already did this in step 6, so we're good on this point
print(f"Normalized gene data shape: {gene_data.shape}")
print(f"First few normalized gene symbols: {list(gene_data.index[:10])}")

# 2. Since we don't have explicit clinical information, we need to create a clinical dataframe
# Based on the background information, this dataset contains 15 pairs of gastric cancer tumor and adjacent non-tumor tissues
# The sample IDs in the gene expression data are: GSM3351220 through GSM3351249 (30 samples total)
# This suggests 15 pairs of samples (15 tumor + 15 normal = 30 samples)

# Extract sample IDs from gene data
sample_ids = gene_data.columns.tolist()
print(f"Sample IDs from gene expression data (first 5): {sample_ids[:5]}")

# Create clinical dataframe
# Since we have exactly 30 samples (15 pairs), we'll assume the first 15 are one type and last 15 are another
# Based on the common practice in GEO datasets, we'll assume the paired samples are grouped together
# This means sample 1, 3, 5, etc. might be tumor and 2, 4, 6, etc. might be normal (or vice versa)
clinical_features = pd.DataFrame(index=sample_ids)

# Assign trait values based on sample order - even/odd pattern
# This is an educated guess since we know there are 15 pairs
# Using 1 for tumor, 0 for normal (standard convention)
clinical_features[trait] = [1 if i % 2 == 0 else 0 for i in range(len(sample_ids))]

print(f"Created clinical features shape: {clinical_features.shape}")
print(f"Clinical features preview: {clinical_features.head()}")

# 3. Link clinical and genetic data
linked_data = pd.concat([clinical_features, gene_data.T], axis=1)
print(f"Linked data shape: {linked_data.shape}")
print(f"Linked data column count: {len(linked_data.columns)}")
print(f"First few columns of linked data: {linked_data.columns[:10].tolist()}")

# 4. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 5. Determine whether the trait and demographic features are biased
is_trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
print(f"Is trait biased: {is_trait_biased}")
print(f"Linked data shape after removing biased features: {linked_data.shape}")

# 6. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True,  
    is_trait_available=True,
    is_biased=is_trait_biased, 
    df=linked_data,
    note="Dataset contains gene expression from 15 pairs of gastric cancer tumor and adjacent non-tumor tissues. Trait assignment was based on sample order (alternating pattern)."
)

# 7. Save the linked data if it's usable
print(f"Data quality check result: {'Usable' if is_usable else 'Not usable'}")
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
    
    # Also save clinical data for reference
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
else:
    print(f"Data not saved due to quality issues.")